## Preprocessing ##

In [1]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import And Read 'charity_data.csv'
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop The Non-Beneficial ID Column 'EIN'
application_df.drop(columns=['EIN'], inplace=True)

In [3]:
# Determine Number Of Unique Values In Each Column
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look At APPLICATION_TYPE Value Counts To Identify And Replace With "Other"
application_types = application_df['APPLICATION_TYPE'].value_counts()
application_types

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose Cutoff Value And Create List Of Application Types To Be Replaced
application_types_to_replace = application_types[application_types < 528].index.tolist()

# Replace In Dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check To Make Sure Replacement Was Successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look At CLASSIFICATION Value Counts To Identify And Replace With "Other"
classification_types = application_df['CLASSIFICATION'].value_counts()
classification_types

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# Look At CLASSIFICATION Value Counts >1
classification_counts_gt_1 = classification_types[classification_types > 1]
classification_counts_gt_1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose Cutoff Value And Create List Of Classifications To Be Replaced
classifications_to_replace = classification_types[classification_types < 100].index.tolist()

# Replace In Dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check To Make Sure Replacement Was Successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
names = application_df['NAME'].value_counts()
print(names[names > 20])

NAME
PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
ARIZONA FEDERATION OF GARDEN CLUBS                   21
ALPHA PHI SIGMA INC                                  21
AMERICAN CHRISTIAN FICTION WRITERS INC               21
FULL GOSPEL BUSINESS MENS FELLOWSHIP IN AMERICA      21
OREGON 4-H FOUNDATION                                21
Name: count, Length: 118, dtype: int64


In [10]:
# Choose Cutoff Value And Create List Of Classifications To Be Replaced
names_to_replace = names[names < 40].index.tolist()

# Replace In Dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name, "Other")

# Check To Make Sure Replacement Was Successful
application_df['NAME'].value_counts()

NAME
Other                                                                    23982
PARENT BOOSTER USA INC                                                    1260
TOPS CLUB INC                                                              765
UNITED STATES BOWLING CONGRESS INC                                         700
WASHINGTON STATE UNIVERSITY                                                492
                                                                         ...  
POP WARNER LITTLE SCHOLARS INC                                              42
UNITED MINE WORKERS OF AMERICA INTERNATIONAL                                41
VETERANS OF FOREIGN WARS OF THE UNITED STATES AUX DEPT OF COLORADO          41
VETERANS OF FOREIGN WARS OF THE U S AUXILIARY DEPARTMENT OF LOUISIANA       41
MODERN QUILT GUILD INC                                                      40
Name: count, Length: 65, dtype: int64

In [11]:
# Convert Categorical Data To Numeric With `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df, dtype=int)

In [12]:
# Split Preprocessed Data Into Features And Target Arrays
y = application_df_dummies['IS_SUCCESSFUL'].values
X = application_df_dummies.drop(columns='IS_SUCCESSFUL').values

# Split Preprocessed Data Into Training And Testing Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [13]:
# Create StandardScaler Instances
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale The Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model ##

In [14]:
# Define The Model - Deep Neural Net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 6

nn = tf.keras.models.Sequential()

# First Hidden Layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="linear"))

# Third Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output Layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check Structure Of The Model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1150      
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 6)                 54        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 1299 (5.07 KB)
Trainable params: 1299 (5.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile The Model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train The Model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5478 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4922 - accuracy: 0.7644
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4852 - accuracy: 0.7659
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4816 - accuracy: 0.7664
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4787 - accuracy: 0.7680
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4764 - accuracy: 0.7684
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4748 - accuracy: 0.7682
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4736 - accuracy: 0.7689
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4723 - accuracy: 0.7692
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4713 - accura

In [17]:
# Evaluate The Model Using The Test Data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4768 - accuracy: 0.7622 - 499ms/epoch - 2ms/step
Loss: 0.4767772853374481, Accuracy: 0.7622157335281372


In [ ]:
# Save And Export Results To An HDF5 File
nn.save("Results/AlphabetSoupCharity__Optimization.h5")